![Shopping trolley in front of a laptop](./iStock-1249219777.jpg)

It's simple to buy any product with a click and have it delivered to your door. Online shopping has been rapidly evolving over the last few years, making our lives easier. But behind the scenes, e-commerce companies face a complex challenge that needs to be addressed. 

Uncertainty plays a big role in how the supply chains plan and organize their operations to ensure that the products are delivered on time. These uncertainties can lead to challenges such as stockouts, delayed deliveries, and increased operational costs.

You work for the Sales & Operations Planning (S&OP) team at a multinational e-commerce company. They need your help to assist in planning for the upcoming end-of-the-year sales. They want to use your insights to plan for promotional opportunities and manage their inventory. This effort is to ensure they have the right products in stock when needed and ensure their customers are satisfied with the prompt delivery to their doorstep.


## The Data

You are provided with a sales dataset to use. A summary and preview are provided below.

# Online Retail.csv

| Column     | Description              |
|------------|--------------------------|
| `'InvoiceNo'` | A 6-digit number uniquely assigned to each transaction |
| `'StockCode'` | A 5-digit number uniquely assigned to each distinct product |
| `'Description'` | The product name |
| `'Quantity'` | The quantity of each product (item) per transaction |
| `'UnitPrice'` | Product price per unit |
| `'CustomerID'` | A 5-digit number uniquely assigned to each customer |
| `'Country'` | The name of the country where each customer resides |
| `'InvoiceDate'` | The day and time when each transaction was generated `"MM/DD/YYYY"` |
| `'Year'` | The year when each transaction was generated |
| `'Month'` | The month when each transaction was generated |
| `'Week'` | The week when each transaction was generated (`1`-`52`) |
| `'Day'` | The day of the month when each transaction was generated (`1`-`31`) |
| `'DayOfWeek'` | The day of the weeke when each transaction was generated <br>(`0` = Monday, `6` = Sunday) |

In [79]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import col, dayofmonth, month, year,  to_date, to_timestamp, weekofyear, dayofweek
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Initialize Spark session
my_spark = SparkSession.builder.appName("SalesForecast").getOrCreate()

# Importing sales data
sales_data = my_spark.read.csv(
    "Online Retail.csv", header=True, inferSchema=True, sep=",")

# Convert InvoiceDate to datetime 
sales_data = sales_data.withColumn("InvoiceDate", to_date(
    to_timestamp(col("InvoiceDate"), "d/M/yyyy H:mm")))

In [80]:
sales_data = sales_data.groupBy("Country", "StockCode", "InvoiceDate", "Year", "Month", "Day", "Week", "DayOfWeek").agg({"Quantity" : "sum", "UnitPrice" : "avg"})
sales_data = sales_data.withColumnRenamed("sum(Quantity)", "Quantity").withColumnRenamed("avg(UnitPrice)", "UnitPrice")
sales_data.show()

+--------------+---------+-----------+----+-----+---+----+---------+---------+--------+
|       Country|StockCode|InvoiceDate|Year|Month|Day|Week|DayOfWeek|UnitPrice|Quantity|
+--------------+---------+-----------+----+-----+---+----+---------+---------+--------+
|United Kingdom|    22912| 2010-01-12|2010|    1| 12|   2|        1|     4.95|       3|
|        France|    22659| 2010-01-12|2010|    1| 12|   2|        1|     1.95|      24|
|United Kingdom|    21544| 2010-01-12|2010|    1| 12|   2|        1|     0.85|      12|
|United Kingdom|    21098| 2010-01-12|2010|    1| 12|   2|        1|     1.25|      16|
|        Norway|    85150| 2010-01-12|2010|    1| 12|   2|        1|     2.55|      12|
|United Kingdom|    22195| 2010-01-12|2010|    1| 12|   2|        1|     1.65|       5|
|United Kingdom|    21668| 2010-01-12|2010|    1| 12|   2|        1|     1.25|      18|
|United Kingdom|    21844| 2010-02-12|2010|    2| 12|   6|        4|     2.95|      12|
|United Kingdom|    22914| 2010-

In [81]:
# Insert the code necessary to solve the assigned problems. Use as many code cells as you need.
train_data = sales_data.filter(sales_data["InvoiceDate"] <= "2011-09-25").select("Country", "StockCode", "InvoiceDate", "Month", "Year", "DayOfWeek", "Day", "Week", "UnitPrice", "Quantity")
pd_daily_train_data = train_data.toPandas()

test_data = sales_data.filter(sales_data["InvoiceDate"] > "2011-09-25").select("Country", "StockCode", "InvoiceDate", "Month", "Year", "DayOfWeek", "Day", "Week", "UnitPrice", "Quantity")
pd_daily_train_data.head()

,Country,StockCode,InvoiceDate,Month,Year,DayOfWeek,Day,Week,UnitPrice,Quantity
0,United Kingdom,22912,2010-01-12,1,2010,1,12,2,4.95,3
1,France,22659,2010-01-12,1,2010,1,12,2,1.95,24
2,United Kingdom,21544,2010-01-12,1,2010,1,12,2,0.85,12
3,United Kingdom,21098,2010-01-12,1,2010,1,12,2,1.25,16
4,Norway,85150,2010-01-12,1,2010,1,12,2,2.55,12


In [82]:
country_indexer = StringIndexer(inputCol="Country", outputCol="CountryIndex").setHandleInvalid("keep")
stock_code_indexer = StringIndexer(inputCol="StockCode", outputCol="StockCodeIndex").setHandleInvalid("keep")

feature_cols = ["CountryIndex", "StockCodeIndex", "Year", "Week"]
assembler = VectorAssembler(inputCols = feature_cols, outputCol = "features")
rf = RandomForestRegressor(featuresCol = "features", labelCol = "Quantity", maxBins=4000)

pipeline = Pipeline(stages=[country_indexer, stock_code_indexer, assembler, rf])
model = pipeline.fit(train_data)
predictions = model.transform(test_data)

24/10/27 09:39:51 WARN DAGScheduler: Broadcasting large task binary with size 1749.8 KiB


24/10/27 09:39:53 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB


In [83]:
test_predictions = predictions.withColumn("prediction", col("prediction").cast("double"))
mae_evaluator = RegressionEvaluator(labelCol="Quantity", predictionCol="prediction", metricName="mae")
mae = mae_evaluator.evaluate(test_predictions)

In [84]:
print(mae)

9.510977576680682


In [85]:
# Getting the weekly sales of all countries
weekly_test_predictions = test_predictions.groupBy("Year", "Week").agg({"prediction": "sum"})

# Finding the quantity sold on the 39 week. 
promotion_week = weekly_test_predictions.filter(col('Year') == 2011).filter(col('Week')==39)

# Storing prediction as quantity_sold_w30
quantity_sold_w39 = int(promotion_week.select("sum(prediction)").collect()[0][0])

In [86]:
print(quantity_sold_w39)

89238
